## Overview
This notebook captures the implementation of training an image classification model to identify if a crop is infected based on the input leaf image. This notebook will be executed in kaggle because of large data size as well as the various options we would like to explore for improving the training accuracy 
- Different Learning Rates : Warm-up LR, Step Decay LR, Cosine Decay LR, Hybrid, Annealing, Adaptive Decay LR
- Different Cost Functions : Use Focal Loss function
- Label Smoothening : Instead of a 0 and 1 for the labels create a continuous distribution of the output

### Model Training

In [185]:
import os
import torch
import warnings
warnings.simplefilter('ignore')
import numpy as np
import torchvision
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn.functional as F
import torchvision.models as models

import imgaug as ia
import imgaug.augmenters as iaa
from collections import Counter

In [172]:
TRAIN_DIR = './images/input/train/'
BATCH_SIZE = 128

#### For GPU

In [22]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [193]:
class CropDataset(Dataset):
    def __init__(self, file_path, transform=None, augment=False):
        xform = transforms.Compose([transforms.ToTensor()]) if transform is None else transform
        self.img_dataset = datasets.ImageFolder(file_path, transform=xform)
        self.aug = augment
        
        ## augumentation initialization
        ia.seed(241)

        self.seq = iaa.Sequential([
            iaa.Fliplr(0.5), # horizontal flips
            iaa.Crop(percent=(0, 0.1)), # random crops
            # Small gaussian blur with random sigma between 0 and 0.5.
            # But we only blur about 50% of all images.
            iaa.Sometimes(
                0.5,
                iaa.GaussianBlur(sigma=(0, 0.5))
            ),
            # Strengthen or weaken the contrast in each image.
            iaa.LinearContrast((0.75, 1.5)),
            # Add gaussian noise.
            # For 50% of all images, we sample the noise once per pixel.
            # For the other 50% of all images, we sample the noise per pixel AND
            # channel. This can change the color (not only brightness) of the
            # pixels.
            iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),
            # Make some images brighter and some darker.
            # In 20% of all cases, we sample the multiplier once per channel,
            # which can end up changing the color of the images.
            iaa.Multiply((0.8, 1.2), per_channel=0.2),
            # Apply affine transformations to each image.
            # Scale/zoom them, translate/move them, rotate them and shear them.
            iaa.Affine(
                scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
                translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
                rotate=(-25, 25),
                shear=(-8, 8)
            )
        ], random_order=True) # apply augmenters in random order

    def __len__(self):
        return len(self.img_dataset)

    def __getitem__(self, idx):
        image, label = self.img_dataset[idx]
        return image, label
    
    # this is an experimental method created to explore how the dunder methods can be modified
    def __repr__(self, idx=0):
        image, label = self.img_dataset[idx]
        plt.imshow(image.permute(1,2,0))
        return str("Display view of the image")
    
    def collate_fn(self, batch):
        images, targets = list(zip(*batch))
        images = torch.stack(images).permute(0 , 2, 3, 1)
        if self.aug: images=self.seq.augment_images(images=images.numpy()) 
        targets = torch.tensor(targets)
        images = torch.tensor(images).permute(0, 3, 1, 2)
        if torch.cuda.is_available():
            targets = targets.to(device)
            images = images.to(device)
        return images, targets

In [170]:
# Here we create our final transformation that would be used before we send the data for training the model
transform = torchvision.transforms.Compose([transforms.ToTensor(),
                                            transforms.Normalize((0.4743617, 0.49847862, 0.4265874 ),
                                                                 (0.21134755, 0.19044809, 0.22679578))]
                                          )
crop_dataset = CropDataset(TRAIN_DIR, transform, True)
print(crop_dataset.img_dataset.class_to_idx)

{'Apple___Apple_scab': 0, 'Apple___Black_rot': 1, 'Apple___Cedar_apple_rust': 2, 'Apple___healthy': 3, 'Blueberry___healthy': 4, 'Cherry_(including_sour)___Powdery_mildew': 5, 'Cherry_(including_sour)___healthy': 6, 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot': 7, 'Corn_(maize)___Common_rust_': 8, 'Corn_(maize)___Northern_Leaf_Blight': 9, 'Corn_(maize)___healthy': 10, 'Grape___Black_rot': 11, 'Grape___Esca_(Black_Measles)': 12, 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)': 13, 'Grape___healthy': 14, 'Peach___Bacterial_spot': 15, 'Peach___healthy': 16, 'Pepper,_bell___Bacterial_spot': 17, 'Pepper,_bell___healthy': 18, 'Potato___Early_blight': 19, 'Potato___Late_blight': 20, 'Potato___healthy': 21, 'Strawberry___Leaf_scorch': 22, 'Strawberry___healthy': 23, 'Tomato___Bacterial_spot': 24, 'Tomato___Early_blight': 25, 'Tomato___Late_blight': 26, 'Tomato___Leaf_Mold': 27, 'Tomato___Septoria_leaf_spot': 28, 'Tomato___Spider_mites Two-spotted_spider_mite': 29, 'Tomato___Target_Spot': 

In [186]:
# define a weighted sampler for the images 
lst_target = []
num_samples = len(crop_dataset.img_dataset)
label = list(crop_dataset.img_dataset.targets)
class_weights = [round(1.0/v,5) for k, v in dict(Counter(crop_dataset.img_dataset.targets)).items()]
weights = [class_weights[label[i]] for i in range(num_samples)]

# Now we can create a loader that will help us load images in batches for training purpose 
sampler = WeightedRandomSampler(weights, num_samples, replacement=True)
crop_loader = DataLoader(dataset=crop_dataset.img_dataset, 
                         batch_size=BATCH_SIZE, 
                         sampler=sampler, 
                         collate_fn=crop_dataset.collate_fn)

Included data from all 10939 samples
Class distribution {22: 315, 3: 357, 33: 335, 0: 301, 7: 287, 32: 320, 19: 311, 29: 336, 11: 330, 12: 297, 17: 338, 9: 305, 25: 338, 30: 326, 21: 313, 24: 330, 8: 325, 6: 289, 10: 305, 26: 314, 23: 317, 20: 369, 31: 326, 16: 315, 1: 312, 27: 341, 14: 335, 15: 319, 2: 316, 18: 323, 4: 328, 5: 333, 28: 306, 13: 327}


In [ ]:
for idx, (image, target) in enumerate(crop_loader):
    lst_target.extend(list(target.numpy()))
    if idx%100 == 0 and idx > 0:
        print(f'Included data from first {idx * BATCH_SIZE} samples')
print(f'Included data from all {num_samples} samples')
print(f'Class distribution {dict(Counter(lst_target))}')
print(f'Total {} records processed')

In [188]:
# Now create a a model that can be trained for disease detection
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 9)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 12, 6)
        self.conv3 = nn.Conv2d(12, 18, 3)
        self.fc1 = nn.Linear(18 * 28 * 28, 4096)
        self.fc2 = nn.Linear(4096, 1024)
        self.fc3 = nn.Linear(1024, 512)
        self.fc4 = nn.Linear(512, 38)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

#### Cross Enropy Loss

In [189]:
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [190]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in net.state_dict():
    print(param_tensor, "\t", net.state_dict()[param_tensor].size())

Model's state_dict:
conv1.weight 	 torch.Size([6, 3, 9, 9])
conv1.bias 	 torch.Size([6])
conv2.weight 	 torch.Size([12, 6, 6, 6])
conv2.bias 	 torch.Size([12])
conv3.weight 	 torch.Size([18, 12, 3, 3])
conv3.bias 	 torch.Size([18])
fc1.weight 	 torch.Size([4096, 14112])
fc1.bias 	 torch.Size([4096])
fc2.weight 	 torch.Size([1024, 4096])
fc2.bias 	 torch.Size([1024])
fc3.weight 	 torch.Size([512, 1024])
fc3.bias 	 torch.Size([512])
fc4.weight 	 torch.Size([38, 512])
fc4.bias 	 torch.Size([38])


In [191]:
# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

Optimizer's state_dict:
state 	 {}
param_groups 	 [{'lr': 0.001, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'params': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]}]


In [23]:
if(torch.cuda.is_available()):
    net.to(device)

#### Training

In [ ]:
for epoch in range(10):  # loop over the dataset multiple times
    running_loss = 0.0
    total = 0
    correct = 0
    net.train()
    for i, data in enumerate(crop_loader, 0):
        if (torch.cuda.is_available()):
            inputs, labels = data[0].to(device), data[1].to(device)
        else:
            inputs, labels = data
        optimizer.zero_grad()
        
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        if i % 100 == 99:
            print(f'Processed {i} batch of 128 images')
            
        if i % 500 == 499:    # print every 2000 mini-batches
            accu=100.*correct/total
            
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 500:.3f} accuracy:{accu:.3f}')
            running_loss = 0.0
print('Finished Training')

In [ ]:
# read the data into dataset

# observe the data

# run the training module 
    # run the augmentation module

# 